In [181]:
import pandas as pd
import numpy as np

class CarboneAlpha:
    """Classe pour représenter un carbone alpha d'une protéine."""
    
    def __init__(self, number, x, y, z):
        self.number = number
        self.x = x
        self.y = y
        self.z = z
    
    def compute_distance(self, other):
        dist = ((other.x - self.x) ** 2 + (other.y - self.y) ** 2 + (other.z - self.z) ** 2) ** 0.5
        return dist

class Template:
    """Classe pour représenter le template utilisé."""

    def __init__(self, file):
        self.structure = self.build_template_from_pdb(file)
        self.length = len(self.structure)

    def build_template_from_pdb(self, filename):
        list_calpha = []
        with open(filename, "r") as pdb :
            for ligne in pdb:
                if ligne.startswith("ATOM") and (ligne[12:16].strip() == "CA"):
                    number = ligne[6:11].strip()
                    x = float(ligne[30:38].strip())
                    y = float(ligne[38:46].strip())
                    z = float(ligne[46:54].strip())
                                       
                    list_calpha.append(CarboneAlpha(number, x, y, z))
        return list_calpha
        
    def build_dist_matrix(self):
        dist_list = []
        
        for i, atom in enumerate(self.structure):
            dist_ligne = []
            for other in (self.structure):
                dist_ligne.append(atom.compute_distance(other))
            dist_list.append(dist_ligne)
            
        dist_matrix = np.array(dist_list)
        return dist_matrix
    
    def __str__(self):
        string = ""
        for i, ca in enumerate(self.structure):
            string += f"position {i}-{ca.number}, coor( {ca.x}, {ca.y}, {ca.z})\n"
        return string

def clean_DOPE_data(filename):
    ca_matrix = []
    
    with open(filename, "r") as dope :
        for ligne in dope:
            if ligne[3:7].strip() == "CA" and ligne[11:14].strip() == "CA":
                ca_matrix.append(ligne.split())
    
    columns = ['res1', 'temp1', 'res2', 'temp2'] + list(np.arange(0.25, 15, 0.5))
    dope_score = pd.DataFrame(ca_matrix, columns = columns) 
    dope_score = dope_score.drop(['temp1', 'temp2'], axis=1)
    
    return dope_score

def get_fasta_sequence(filename):
    sequence = ""
    with open(filename, "r") as fasta:
        for ligne in fasta:
            if ligne.startswith(">"):
                continue
            sequence += ligne.strip()
    return sequence

class DynamicMatrix:    
    def __init__(self, lines, columns, gap):
        self.matrix = np.zeros((lines, columns))
        self.lines = lines
        self.columns = columns
        self.gap = gap

    def initialize_matrix(self, first_val, start, end):
        if (start[0] < 0) or (start[1] < 0):
            raise ValueError("Start of initialization out of matrix.")
        if (end[0] >= self.lines) or (end[1] >= self.columns):
            raise ValueError("End of initialization out of matrix.")
        
        # Première case
        self.matrix[start[0], start[1]] = first_val
        
        # Remplissage de la première colonne jusqu'à la limite
        for i in range(start[0] + 1, end[0] + 1):
            self.matrix[i, start[1]] = self.matrix[i - 1, start[1]] + self.gap

        # Remplissage de la première ligne jusqu'à la limite
        for j in range(start[1] + 1, end[1] + 1):
            self.matrix[start[0], j] = self.matrix[start[0], j - 1] + self.gap

class LowLevelMatrix(DynamicMatrix):
    aa_codes = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS',
    'Q': 'GLN', 'E': 'GLU', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
    'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO',
    'S': 'SER', 'T': 'THR', 'W': 'TRP', 'Y': 'TYR', 'V': 'VAL'
    }
    
    def __init__(self, gap, frozen, distance, dope, sequence):
        lines = len(sequence)
        columns = len(distance)
        
        DynamicMatrix.__init__(self, lines, columns, gap)

        # Vérification du blocage de la case
        if (frozen['seq_id'] >= lines) or (frozen['seq_id'] < 0):
            raise ValueError("Frozen line index out of matrix.")
        if (frozen['pos_id'] >= columns) or (frozen['pos_id'] < 0):
            raise ValueError("Frozen column index out of matrix")

        # Récupération du résidu fixé
        frozen['seq_res'] = sequence[frozen['seq_id']]
        
        self.frozen = frozen
        self.distance = distance
        self.dope = dope
        self.sequence = sequence

    def round_distance(self, dist):
        # arrondi au quart le plus proche
        rounded_value = round(dist * 4) / 4
        
        # ne garde que 0.25 ou 0.75
        decimal = rounded_value % 1
        if decimal == 0.0:
            return rounded_value + 0.25
        elif decimal == 0.5:
            return rounded_value + 0.25
        else:
            return rounded_value
    
    def get_score(self, i, j):
        # Cas du résidu bloqué avec sa propre position
        #if (i == self.frozen["seq_id"] or j == self.frozen["pos_id"]):
            #return 0
        
        dist = self.distance[self.frozen["pos_id"], j]
        closest_dist = self.round_distance(dist)
        score = self.dope.loc[(self.dope['res1'] == self.aa_codes[self.frozen['seq_res']]) & 
                              (self.dope['res2'] == self.aa_codes[self.sequence[i]]), 
                              closest_dist]
        
        return float(score.values[0])
    
    def fill_matrix(self):
        # Partie supérieure gauche
        self.initialize_matrix(self.get_score(0, 0), [0, 0], 
                               [self.frozen['seq_id'] - 1, self.frozen['pos_id'] - 1])
        
        for i in range(1, self.frozen['seq_id']):
            for j in range(1, self.frozen['pos_id']):
                score = self.get_score(i, j)
                self.matrix[i, j] = min(self.matrix[i - 1, j - 1] + score,
                                        self.matrix[i - 1, j] + self.gap,
                                        self.matrix[i, j - 1] + self.gap
                                       )

        # Case fixée
        self.matrix[self.frozen['seq_id'], self.frozen['pos_id']] = self.matrix[self.frozen['seq_id'] - 1, self.frozen['pos_id'] - 1]
        
        # Partie inférieure droite (si elle existe)
        if (self.frozen['pos_id'] == self.columns - 1 or self.frozen['seq_id'] == self.lines - 1):
            max_score = self.matrix[self.frozen['seq_id'], self.frozen['pos_id']]

        else:
            self.initialize_matrix(self.matrix[self.frozen['seq_id'], self.frozen['pos_id']],
                                   [self.frozen['seq_id'] + 1, self.frozen['pos_id'] + 1],
                                   [self.lines - 1, self.columns - 1])
    
            for i in range(self.frozen['seq_id'] + 1, self.lines):
                for j in range(self.frozen['pos_id'] + 1, self.columns):
                    score = self.get_score(i, j)
                    self.matrix[i, j] = min(self.matrix[i - 1, j - 1] + score,
                                            self.matrix[i - 1, j] + self.gap,
                                            self.matrix[i, j - 1] + self.gap
                                           )
    
            max_score = self.matrix[self.lines - 1, self.columns - 1]
        return max_score

class HighLevelMatrix(DynamicMatrix):
    def __init__(self, gap, query, template, dope):
        distance = template.build_dist_matrix()
        lines = len(query)
        columns = len(distance)

        DynamicMatrix.__init__(self, lines, columns, gap)

        self.sequence = query
        self.distance = distance
        self.dope = dope

        self.get_score_matrix()

    def get_score_matrix(self):
        self.score_matrix = np.zeros((self.lines, self.columns))
        for i in range(self.lines):
            for j in range(self.columns):
                frozen = {'seq_id': i, 'pos_id': j}
                low_level = LowLevelMatrix(self.gap, frozen, self.distance, self.dope, self.sequence)
                self.score_matrix[i, j] =  low_level.fill_matrix()

    def get_score(self, i, j):
        score = self.score_matrix[i, j]
        return score

    def fill_matrix(self):
        # Initialisation
        self.initialize_matrix(self.get_score(0, 0), [0, 0], 
                               [self.lines - 1, self.columns - 1])
        
        # Remplissage
        for i in range(1, self.lines):
            for j in range(1, self.columns):
                score = self.get_score(i, j)
                self.matrix[i, j] = min(self.matrix[i - 1, j - 1] + score,
                                        self.matrix[i - 1, j] + self.gap,
                                        self.matrix[i, j - 1] + self.gap
                                       )
        max_score = self.matrix[self.lines - 1, self.columns - 1]

        return max_score

    def get_alignment(self):
        structure_align = []
        sequence_align = []
        
        i = self.lines - 1
        j = self.columns - 1
        while not ((i == 0) and (j == 0)):
            print(i, j)
            square = self.matrix[i, j]
            score = self.score_matrix[i, j]
            # Match
            if (square == self.matrix[i - 1, j - 1] + score):
                print("match")
                structure_align.insert(0, j + 1)
                sequence_align.insert(0, self.sequence[i])
                i = i - 1
                j = j - 1
            # Gap
            else:
                if (square == self.matrix[i - 1, j] + self.gap):
                    print("gap structure")
                    structure_align.insert(0, '-')
                    sequence_align.insert(0, self.sequence[i])
                    i = i - 1
                elif (square == self.matrix[i, j - 1] + self.gap):
                    print("gap sequence")
                    structure_align.insert(0, j + 1)
                    sequence_align.insert(0, '-')
                    j = j - 1

        return ''.join(sequence_align), ''.join(str(x) for x in structure_align)

In [182]:
# Séquence 'query'
FASTA_FILE = "../data/5AWL.fasta"
QUERY = get_fasta_sequence(FASTA_FILE)

# Structure 'template'
PDB_FILE = "../data/5awl.pdb"
TEMPLATE = Template(PDB_FILE)

# Matrice DOPE
DOPE_FILE = "../data/dope.par"
DOPE_MATRIX = clean_DOPE_data(DOPE_FILE)

# Information(s) supplémentaire(s)
GAP = 0

In [183]:
print(LOW_TEST.lines)

10


In [188]:
DIST_MATRIX = TEMPLATE.build_dist_matrix()
FROZEN = {'seq_id': 0, 'pos_id': 0}
SEQUENCE = "YYDPETGTWY"
GAP = 0
    
LOW_TEST = LowLevelMatrix(GAP, FROZEN, DIST_MATRIX, DOPE_MATRIX, SEQUENCE)
MAX_SCORE = LOW_TEST.fill_matrix()
print(MAX_SCORE)
print(LOW_TEST.matrix)

-3.0
[[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.   -1.41 -1.41 -1.41 -1.41 -1.41 -1.41 -1.41 -1.41 -1.41]
 [ 0.   -1.69 -1.69 -1.69 -1.69 -1.69 -1.69 -1.69 -1.85 -1.85]
 [ 0.   -1.69 -1.85 -1.88 -1.88 -1.88 -1.88 -1.88 -2.1  -2.26]
 [ 0.   -1.69 -1.85 -1.88 -1.92 -1.92 -1.92 -1.92 -2.35 -2.57]
 [ 0.   -1.69 -1.85 -1.9  -1.92 -1.95 -1.97 -1.97 -2.35 -2.77]
 [ 0.   -1.69 -1.85 -1.9  -1.92 -1.95 -1.98 -1.98 -2.39 -2.77]
 [ 0.   -1.69 -1.85 -1.9  -1.94 -1.95 -2.   -2.   -2.4  -2.81]
 [ 0.   -1.69 -1.85 -1.97 -1.99 -1.99 -2.07 -2.07 -2.48 -2.88]
 [ 0.   -1.69 -1.85 -1.97 -1.99 -2.06 -2.07 -2.07 -2.59 -3.  ]]


In [158]:
# Si pas de message, ça tourne
for i in range(10):
    for j in range(10):
        FROZEN = {'seq_id': i, 'pos_id': j}
            
        LOW_TEST = LowLevelMatrix(GAP, FROZEN, DIST_MATRIX, DOPE_MATRIX, SEQUENCE)
        MAX_SCORE = LOW_TEST.fill_matrix()

In [185]:
# Algorithme principal
HIGH_LEVEL = HighLevelMatrix(GAP, QUERY, TEMPLATE, DOPE_MATRIX)
MAX_SCORE = HIGH_LEVEL.fill_matrix()
print(MAX_SCORE)
ALIGN_SEQ, ALIGN_STRUCT = HIGH_LEVEL.get_alignment()
print(ALIGN_SEQ)
print(ALIGN_STRUCT)

-45.35
9 9
match
8 8
match
7 7
match
6 6
match
5 5
match
4 4
match
3 3
match
2 2
match
1 1
match
YDPETGTWY
2345678910


In [187]:
print(HIGH_LEVEL.score_matrix)

[[-3.   -3.52 -3.92 -2.99 -2.41 -2.52 -2.08 -1.85 -1.69  0.  ]
 [-3.   -4.93 -3.96 -3.01 -2.39 -2.59 -2.1  -1.71 -2.21 -0.52]
 [-3.11 -5.16 -5.73 -4.6  -3.91 -4.41 -3.81 -3.38 -4.03 -2.13]
 [-2.94 -4.91 -5.48 -5.5  -4.86 -4.97 -4.76 -4.33 -4.59 -2.75]
 [-2.54 -4.83 -5.27 -4.27 -4.36 -4.71 -4.77 -4.33 -4.08 -2.49]
 [-2.7  -4.5  -4.78 -4.66 -4.53 -5.05 -5.44 -4.99 -4.26 -2.61]
 [-2.39 -4.07 -4.16 -4.32 -3.98 -4.64 -5.   -4.72 -4.21 -2.57]
 [-1.93 -3.67 -3.86 -4.05 -4.21 -4.62 -5.41 -4.67 -4.31 -2.85]
 [-1.45 -2.9  -3.02 -3.66 -3.72 -4.05 -4.9  -4.39 -4.09 -2.74]
 [ 0.   -1.41 -1.73 -2.04 -2.23 -2.61 -3.46 -3.18 -2.77 -3.02]]


In [175]:
# Algorithme principal
test_seq = "WWWWWWWWWW"

HIGH_LEVEL = HighLevelMatrix(GAP, test_seq, TEMPLATE, DOPE_MATRIX)
MAX_SCORE = HIGH_LEVEL.fill_matrix()
print(MAX_SCORE)
ALIGN_SEQ, ALIGN_STRUCT = HIGH_LEVEL.get_alignment()
print(ALIGN_SEQ)
print(ALIGN_STRUCT)

-35.449999999999996
9 9
match
8 8
match
7 7
match
6 6
match
5 5
match
4 4
match
3 3
match
2 2
match
1 1
match
WWWWWWWWW
2345678910
